<a href="https://www.kaggle.com/code/quctngngvng/nlp-umls-cadec?scriptVersionId=121112347" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [3]:
from tensorflow.keras.utils import to_categorical

In [4]:
# Detect hardware, set appropriate distribution strategy (GPU/TPU)
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

TPU=True
strategy = auto_select_accelerator()

Running on 1 replicas


In [5]:
# Convert integers to one-hot encoding
# one_hot = to_categorical(label_ints, num_classes=len(label_map))

# # Print the one-hot encoding
# print(one_hot)

In [6]:
# END test**

In [7]:
df0 = pd.read_csv("/kaggle/input/nlp-umls/cadec_mcn_folds/train_0.csv",sep = '\t')
df1 = pd.read_csv("/kaggle/input/nlp-umls/cadec_mcn_folds/train_1.csv",sep = '\t')
df2 = pd.read_csv("/kaggle/input/nlp-umls/cadec_mcn_folds/train_2.csv",sep = '\t')
df3 = pd.read_csv("/kaggle/input/nlp-umls/cadec_mcn_folds/train_3.csv",sep = '\t')
df4 = pd.read_csv("/kaggle/input/nlp-umls/cadec_mcn_folds/train_4.csv",sep = '\t')


In [8]:
df4.head(3)

,memory fog,386807006
0,aching (like the flu),82991003
1,lost a lot of muscle and replacedit with fat,88092000
2,elevated liver enzymes,166643006


In [9]:
df1=df1.rename(columns = {'pain in the shoulder':'pain in upper back (over left shoulder blade)','45326000':'279038004'})
df2=df2.rename(columns = {'severe pain in arm':'pain in upper back (over left shoulder blade)','102556003':'279038004'})
df3=df3.rename(columns = {'numbing of my feet':'pain in upper back (over left shoulder blade)','309538000':'279038004'})
df4=df4.rename(columns = {'memory fog':'pain in upper back (over left shoulder blade)','386807006':'279038004'})


In [10]:
df = df0
df.append(df2)
df.append(df2)
df.append(df3)
df.append(df4)

,pain in upper back (over left shoulder blade),279038004
0,gastritis,162076009
1,hard to get out of bed in the morning,301676004
2,very weak,13791008
3,uterine cramps,29542008
4,foggy-headedness,419723007
...,...,...
1778,losing muscle,88092000
1779,couldn't remember where i left my dog 10 minut...,386807006
1780,weakness in both legs,309774006
1781,blood in my stools,405729008


In [11]:
label_map = {}

In [12]:
label_map

{}

In [13]:
for i in set(df['279038004'].values):
    label_map[i] = len(label_map)

In [14]:
X_train = df['pain in upper back (over left shoulder blade)'].values
Y_train = df['279038004'].values
Y_train = [label_map[label] for label in Y_train]
Y_train = to_categorical(Y_train, num_classes=len(label_map))


In [15]:
label_output = [i for i in label_map.keys() ]

In [16]:
pip install scikit-learn

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [17]:
df['279038004'].value_counts()

68962001     59
162076009    44
62507009     35
55300003     31
228158008    29
             ..
230690007     4
426899007     4
67849003      4
162352007     4
279038004     3
Name: 279038004, Length: 181, dtype: int64

In [18]:
# from sklearn.model_selection import train_test_split
# X_train, x_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size=0.33, random_state=2)

In [19]:
X_train.shape

(1654,)

# Tải mô hinh bert
encoder text

In [20]:
# # Load the BERT model from Hub
# bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# Khoi tao cac lop bert

Tạo lớp đầu vào bằng tf.keras.layers.Input
preprocessed_textlàm đầu vào cho lớp này.
bert_encodernăng sẽ chuyển đổi văn bản được xử lý trước thành các vectơ nhúng.
outputssẽ được đưa vào các lớp mạng thần kinh.


In [21]:
X_train

array(['gastritis', 'hard to get out of bed in the morning', 'very weak',
       ..., 'severe headache', 'neurophathy', 'calves muscle pain in'],
      dtype=object)

In [22]:
model=0

In [23]:
from sklearn.utils.class_weight import compute_class_weight
# Calculate class weights
#class_weights = class_weight.compute_class_weight(np.unique(np.argmax(Y_train, axis=1)), np.argmax(Y_train, axis=1))
class_weights = compute_class_weight(class_weight = "balanced"
                                     , classes= np.unique(np.argmax(Y_train, axis=1))
                                     , y= np.argmax(Y_train, axis=1))
# Normalize class weights
class_weights = {i : class_weights[i] / np.sum(class_weights) for i in range(len(class_weights))}

In [24]:
len(class_weights)

181

In [25]:
# # Load pre-trained BERT model from TensorFlow Hub
bert_layer =  hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=True)
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [26]:

# Define the model architecture
inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
input_encoder = bert_preprocess(inputs)
preprocessed_inputs = bert_layer(input_encoder)['pooled_output']
# output_layer = tf.keras.layers.Dense(384, activation='relu')(preprocessed_inputs)
# output_layer = tf.keras.layers.Dense(250, activation='relu')(output_layer)
output_layer = tf.keras.layers.Dense(181, activation='softmax')(preprocessed_inputs)
model = tf.keras.models.Model(inputs, outputs=output_layer)

# Compile the model with categorical crossentropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
             #loss_weights=class_weights)
#model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),loss = )
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [27]:
from sklearn.model_selection import KFold

In [28]:
kfold = KFold(n_splits=10, shuffle=True)
fold_idx = 1
batch_size = 1
accuracy_list=[]
loss_list = [] 

for train_ids, val_ids in kfold.split(X_train, Y_train):

    print("Bắt đầu train Fold ", fold_idx)

    # Train model
    model.fit(X_train[train_ids], Y_train[train_ids],
              batch_size=batch_size,
              epochs=5,class_weight = class_weights)

    # Test và in kết quả
    scores = model.evaluate(X_train[val_ids],Y_train[val_ids], verbose=0)
    print("Đã train xong Fold ", fold_idx)
    
     
    # Thêm thông tin accuracy và loss vào list
    accuracy_list.append(scores[1] * 100)
    loss_list.append(scores[0])

    # Sang Fold tiếp theo
    fold_idx = fold_idx + 1

Bắt đầu train Fold  1
Epoch 1/5
1488/1488 [==============================] - 113s 64ms/step - loss: 0.0195 - accuracy: 0.0074
Epoch 2/5
1488/1488 [==============================] - 96s 64ms/step - loss: 0.0168 - accuracy: 0.1317
Epoch 3/5
1488/1488 [==============================] - 95s 64ms/step - loss: 0.0124 - accuracy: 0.4610
Epoch 4/5
1488/1488 [==============================] - 95s 64ms/step - loss: 0.0089 - accuracy: 0.6647
Epoch 5/5
1488/1488 [==============================] - 96s 64ms/step - loss: 0.0062 - accuracy: 0.7910
Đã train xong Fold  1
Bắt đầu train Fold  2
Epoch 1/5
1488/1488 [==============================] - 95s 64ms/step - loss: 0.0045 - accuracy: 0.8542
Epoch 2/5
1488/1488 [==============================] - 95s 64ms/step - loss: 0.0028 - accuracy: 0.8931
Epoch 3/5
1488/1488 [==============================] - 95s 64ms/step - loss: 0.0018 - accuracy: 0.9274
Epoch 4/5
1488/1488 [==============================] - 95s 64ms/step - loss: 0.0012 - accuracy: 0.9509
Epoch 

In [29]:
print('* Chi tiết các fold')
for i in range(0, len(accuracy_list)):
  print(f'> Fold {i+1} - Loss: {loss_list[i]} - Accuracy: {accuracy_list[i]}%')

print('* Đánh giá tổng thể các folds:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')
print(f'> Loss: {np.mean(loss_list)}')

* Chi tiết các fold
> Fold 1 - Loss: 2.0249927043914795 - Accuracy: 67.46987700462341%
> Fold 2 - Loss: 0.29258477687835693 - Accuracy: 93.37349534034729%
> Fold 3 - Loss: 0.15836800634860992 - Accuracy: 95.783132314682%
> Fold 4 - Loss: 0.12545233964920044 - Accuracy: 97.59036302566528%
> Fold 5 - Loss: 0.17933493852615356 - Accuracy: 96.36363387107849%
> Fold 6 - Loss: 0.22226254642009735 - Accuracy: 95.75757384300232%
> Fold 7 - Loss: 0.03872423246502876 - Accuracy: 99.39393997192383%
> Fold 8 - Loss: 0.013665440492331982 - Accuracy: 99.39393997192383%
> Fold 9 - Loss: 0.011159120127558708 - Accuracy: 99.39393997192383%
> Fold 10 - Loss: 0.0780043974518776 - Accuracy: 98.18181991577148%
* Đánh giá tổng thể các folds:
> Accuracy: 94.27017152309418 (Độ lệch +- 9.127033960643537)
> Loss: 0.3144548502750695


In [30]:
# Train the model
# model.fit(X_train, Y_train, validation_data=(x_val, y_val), epochs=10,shuffle=True, batch_size=1)

# label_output[int(np.argmax(y_test, axis=1))]

In [ ]:
df_test = pd.read_csv('/kaggle/input/nlp-umls/cadec_mcn_folds/test_1.csv',sep = '\t')
df_test.head(3)

In [32]:
X_test = df_test['gastric problems'].values
Y_test = df_test['162076009'].values

In [33]:
Y_predict = model.predict(X_test)
Y_predict_number = np.argmax(Y_predict, axis=1)
Y_predict_number =[label_output[i] for i in Y_predict_number] 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_predict_number)

15/15 [==============================] - 6s 336ms/step


0.9934497816593887

In [34]:
df_test3 = pd.read_csv('/kaggle/input/nlp-umls/cadec_mcn_folds/test_3.csv',sep = '\t')
X_test3 = df_test3['pains'].values
Y_test3 = df_test3['22253000'].values
Y_predict3 = model.predict(X_test3)
Y_predict_number3 = np.argmax(Y_predict3, axis=1)
Y_predict_number3 =[label_output[i] for i in Y_predict_number3] 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test3, Y_predict_number3)

13/13 [==============================] - 4s 311ms/step


0.9949494949494949

In [39]:
data_sample = ["i can't stand or walk for any lengths of time"]

In [41]:
 label_output[np.argmax(model.predict(data_sample))]

1/1 [==============================] - 0s 80ms/step


8510008

In [43]:
model.save("NLPBert_GPU_T4.h5")

In [44]:
cd /kaggle/working

/kaggle/working


In [51]:
from IPython.display import FileLink 
FileLink(d'/kaggle/working/NLPBert_GPU_T4.h5')

SyntaxError: invalid syntax (2628698473.py, line 2)